In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scipy
import os.path
import pandas
from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from  mefs_from_simple_dispatch import generateMefs
from  mefs_from_simple_dispatch import plotDispatch
from simple_dispatch import generatorDataShort

In [57]:
run_year = 2019
egrid_data_xlsx = 'egrid2019_data.xlsx'
nerc_region = 'WECC'

In [24]:
gd_short = pickle.load(open('generator_data_short_%s_%s.obj'%(nerc_region, str(run_year)), 'rb'))

# 3.  Retirements

In [4]:
gd_short.df.head()

,ba,co21,co210,co211,co212,co213,co214,co215,co216,co217,...,fuel_price45,fuel_price46,fuel_price47,fuel_price48,fuel_price49,fuel_price50,fuel_price51,fuel_price52,min_out_multiplier,min_out
0,CISO,440.027117,443.418463,438.040757,438.040757,438.040757,438.040757,438.040757,438.040757,447.237754,...,2.923,2.923,2.923,3.985,3.985,3.985,3.985,3.985,0.5,49.857948
1,CISO,575.900478,0.000000,599.390089,0.000000,0.000000,0.000000,0.000000,0.000000,447.237754,...,2.49124,2.49124,2.49124,2.77522,2.77522,2.77522,2.77522,2.77522,0.5,19.474673
2,CISO,435.268020,432.974659,599.390089,0.000000,0.000000,0.000000,0.000000,0.000000,447.237754,...,2.893,2.893,2.893,3.887,3.887,3.887,3.887,3.887,0.5,19.345360
3,CISO,546.510324,559.862743,643.276636,502.278095,491.061008,469.410508,616.815110,467.767266,577.299545,...,2.923,2.923,2.923,3.985,3.985,3.985,3.985,3.985,0.5,21.627335
4,CISO,439.170204,559.862743,643.276636,502.278095,491.061008,469.410508,616.815110,467.767266,577.299545,...,2.923,2.923,2.923,3.985,3.985,3.985,3.985,3.985,0.5,55.218727


In [5]:
egrid_unt = pandas.read_excel(egrid_data_xlsx, 'UNT'+'19', skiprows=[0])

In [6]:
egrid_gen = pandas.read_excel(egrid_data_xlsx, 'GEN'+'19', skiprows=[0])
egrid_plnt = pandas.read_excel(egrid_data_xlsx, 'PLNT'+'19', skiprows=[0])

In [7]:
egrid_unt['orispl_unit'] = egrid_unt.ORISPL.map(str) + '_' + egrid_unt.UNITID.map(str)

In [43]:
inds_2019_ret = egrid_gen.loc[(egrid_gen['ORISPL'].isin(gd_short.df['orispl']))&(egrid_gen['GENYRRET']==2019)].index
orispl_unit_2019_ret = egrid_gen.loc[inds_2019_ret, 'ORISPL'].map(str) + '_' + egrid_gen.loc[inds_2019_ret, 'GENID'].map(str)

In [44]:
# ## From: https://www.wecc.org/Administrative/06_Brown_Retirements_October%202020.pdf
unit_drops = pd.DataFrame({'WECC Name':['H Wilson Sundt Generating Station', 'H Wilson Sundt Generating Station'], 'orispl':[126, 126], 'orispl_unit':['126_1', '126_2'], 'retirement_year':[2020,2020]})
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Colstrip', '6076', '6076_1', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Colstrip', '6076', '6076_2', 2020]

In [45]:
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Ontario Mill', '10427', '10427_GEN1', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Fullerton Mill Cogeneration', '58083', '58083_GTG1', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Inland Empire Energy Center', '55853', '55853_1', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Inland Empire Energy Center', '55853', '55853_2', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['AES Alamitos LLC', '315', '315_1', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['AES Alamitos LLC', '315', '315_2', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['AES Alamitos LLC', '315', '315_6', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Escalante', '87', '87_1', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Centralia Complex', '3845', '3845_BW21', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Cholla', '113', '113_4', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Ellwood Generating Station', '8076', '8076_01', 2020]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Boardman (OR)', '6106', '6106_1SG', 2021]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Redondo Beach', '356', '356_5', 2021]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Redondo Beach', '356', '356_6', 2021]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Redondo Beach', '356', '356_8', 2021]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Fort Churchill', '2330', '2330_2', 2021]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Comanche (CO)', '470', '470_1', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Rio Grande', '2444', '2444_6', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Rio Grande', '2444', '2444_7', 2022]

In [46]:
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['San Juan Generating Station', '2451', '2451_1', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['San Juan Generating Station', '2451', '2451_4', 2022]


In [47]:
# to run when complete list:


unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Newman', '3456', '3456_1', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Newman', '3456', '3456_2', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Oakland', '6211', '6211_GEN1', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Oakland', '6211', '6211_GEN2', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Oakland', '6211', '6211_GEN3', 2022]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 1 Garnet Valley', '54350', '54350_AUX', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 1 Garnet Valley', '54350', '54350_GTA', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 1 Garnet Valley', '54350', '54350_GTB', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 1 Garnet Valley', '54350', '54350_GTC', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 2 Black Mtn', '54349', '54349_AUX', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 2 Black Mtn', '54349', '54349_GTA', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 2 Black Mtn', '54349', '54349_GTB', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Nevada Cogen Assoc 2 Black Mtn', '54349', '54349_GTC', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Haynes', '400', '400_1', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Haynes', '400', '400_2', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Martin Drake', '492', '492_6', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Martin Drake', '492', '492_7', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['AES Alamitos LLC', '315', '315_3', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['AES Alamitos LLC', '315', '315_4', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['AES Alamitos LLC', '315', '315_5', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Ormond Beach Generating Station', '350', '350_1', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Ormond Beach Generating Station', '350', '350_2', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Huntington Beach', '335', '335_2', 2023]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Diablo Canyon', '6099', '6099_1', 2024]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Scattergood', '404', '404_1', 2024]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Scattergood', '404', '404_2', 2024]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Cholla', '113', '113_1', 2024]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Cholla', '113', '113_3', 2024]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Diablo Canyon', '6099', '6099_2', 2025]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Fort Churchill', '2330', '2330_1', 2025]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Centralia Complex', '3845', '3845_BW22', 2025]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['North Valmy', '8224', '8224_1', 2025]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['North Valmy', '8224', '8224_2', 2025]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Comanche (CO)', '470', '470_2', 2025]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Craig (CO)', '6021', '6021_C1', 2025]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Newman', '3456', '3456_4', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Valmont', '477', '477_6', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['J M Shaffer', '50707', '50707_S001', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['J M Shaffer', '50707', '50707_S002', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Fruita', '471', '471_1', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Alamosa', '464', '464_CT1', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Alamosa', '464', '464_CT2', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Newman', '3456', '3456_3', 2026]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Intermountain', '6481', '6481_1SGA', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Intermountain', '6481', '6481_2SGA', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Springerville', '8223', '8223_1', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Dave Johnston', '4158', '4158_BW41', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Dave Johnston', '4158', '4158_BW42', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Dave Johnston', '4158', '4158_BW43', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Dave Johnston', '4158', '4158_BW44', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Rio Grande', '2444', '2444_8', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Cherokee (CO)', '469', '469_4', 2027]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Craig (CO)', '6021', '6021_C2', 2028]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Craig (CO)', '6021', '6021_C3', 2028]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Naughton', '4162', '4162_1', 2029]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Naughton', '4162', '4162_2', 2029]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Rawhide', '6761', '6761_101', 2029]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Ray Nixon', '8219', '8219_1', 2030]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Four Corners', '2442', '2442_4', 2031]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Four Corners', '2442', '2442_5', 2031]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Springerville', '8223', '8223_2', 2032]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Coronado', '6177', '6177_U1B', 2032]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Coronado', '6177', '6177_U2B', 2032]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Jim Bridger', '8066', '8066_BW71', 2037]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Jim Bridger', '8066', '8066_BW72', 2037]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Jim Bridger', '8066', '8066_BW73', 2037]
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Jim Bridger', '8066', '8066_BW74', 2037]



In [48]:
unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit', 'retirement_year']] = ['Greenleaf Unit Two', '10350', '10350_1', 2020]

In [49]:
list(orispl_unit_2019_ret.values)

['126_ST1',
 '126_ST2',
 '4941_NAV1',
 '4941_NAV2',
 '4941_NAV3',
 '116_1',
 '116_2',
 '315_1',
 '315_2',
 '315_6',
 '335_1',
 '356_7',
 '7693_1',
 '10350_GEN1',
 '10350_GEN2',
 '527_1',
 '527_2',
 '527_3',
 '527_ST4']

In [50]:
orispl_unit_2019_ret_edited = ['126_1', '126_2', '4941_1', '4941_2', '4941_3', '116_1', '116_2', 
                               '315_1', '315_2', '315_6', '335_1', '356_7', '7693_1', '10350_1', '527_1']

In [51]:
egrid_unt[egrid_unt['orispl_unit'].isin(orispl_unit_2019_ret_edited)]

,SEQUNT19,YEAR,PSTATABB,PNAME,ORISPL,UNITID,PRMVR,UNTOPST,CAMDFLAG,PRGCODE,...,NOXANSRC,NOXOZSRC,SO2SRC,CO2SRC,HGSRC,SO2CTLDV,NOXCTLDV,HGCTLDV,UNTYRONL,orispl_unit
1481,1482,2019,AZ,Irvington Generating Station,126,1,ST,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,NaN,NaN,1958.0,126_1
1482,1483,2019,AZ,Irvington Generating Station,126,2,ST,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,NaN,NaN,1960.0,126_2
1521,1522,2019,AZ,Navajo Generating Station,4941,1,ST,OP,Yes,"ARP,MATS",...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,WLS,"LNC3,O",Yes,1974.0,4941_1
1522,1523,2019,AZ,Navajo Generating Station,4941,2,ST,OP,Yes,"ARP,MATS",...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,WLS,"LNC3,O",Yes,1975.0,4941_2
1523,1524,2019,AZ,Navajo Generating Station,4941,3,ST,OP,Yes,"ARP,MATS",...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,WLS,"LNC3,O",Yes,1976.0,4941_3
1534,1535,2019,AZ,Ocotillo Power Plant,116,1,ST,OP,Yes,ARP,...,EPA/CAMD,Estimated using emissions factor,EPA/CAMD,EPA/CAMD,NaN,NaN,NaN,NaN,1960.0,116_1
1535,1536,2019,AZ,Ocotillo Power Plant,116,2,ST,OP,Yes,ARP,...,EPA/CAMD,Estimated using emissions factor,EPA/CAMD,EPA/CAMD,NaN,NaN,NaN,NaN,1960.0,116_2
1677,1678,2019,CA,AES Alamitos,315,1,ST,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,SR,NaN,1956.0,315_1
1678,1679,2019,CA,AES Alamitos,315,2,ST,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,SR,NaN,1957.0,315_2
1682,1683,2019,CA,AES Alamitos,315,6,ST,OP,Yes,ARP,...,EPA/CAMD,EPA/CAMD,EPA/CAMD,EPA/CAMD,NaN,NaN,SR,NaN,1966.0,315_6


In [52]:
for i in egrid_unt[egrid_unt['orispl_unit'].isin(orispl_unit_2019_ret_edited)].index:
    unit_drops.loc[len(unit_drops), ['WECC Name', 'orispl', 'orispl_unit']] = egrid_unt.loc[i, ['PNAME', 'ORISPL', 'orispl_unit']].values
    unit_drops.loc[len(unit_drops)-1, 'retirement_year'] = 2019

In [53]:
unit_drops

,WECC Name,orispl,orispl_unit,retirement_year
0,H Wilson Sundt Generating Station,126,126_1,2020.0
1,H Wilson Sundt Generating Station,126,126_2,2020.0
2,Colstrip,6076,6076_1,2020.0
3,Colstrip,6076,6076_2,2020.0
4,Ontario Mill,10427,10427_GEN1,2020.0
...,...,...,...,...
103,AES Huntington Beach,335,335_1,2019.0
104,AES Redondo Beach,356,356_7,2019.0
105,Anaheim Combustion Turbine,7693,7693_1,2019.0
106,Greenleaf One,10350,10350_1,2019.0


In [55]:
unit_drops.to_csv('scheduled_retirements_2019.csv')

In [297]:
unit_drops = pd.read_csv('scheduled_retirements_2019.csv', index_col=0)

In [298]:
unit_drops

,WECC Name,orispl,orispl_unit,retirement_year
0,H Wilson Sundt Generating Station,126.0,126_1,2020.0
1,H Wilson Sundt Generating Station,126.0,126_2,2020.0
2,Colstrip,6076.0,6076_1,2020.0
3,Colstrip,6076.0,6076_2,2020.0
4,Ontario Mill,10427.0,10427_GEN1,2020.0
...,...,...,...,...
103,AES Huntington Beach,335.0,335_1,2019.0
104,AES Redondo Beach,356.0,356_7,2019.0
105,Anaheim Combustion Turbine,7693.0,7693_1,2019.0
106,Greenleaf One,10350.0,10350_1,2019.0


In [314]:
egrid_gen.columns

Index(['SEQGEN19', 'YEAR', 'PSTATABB', 'PNAME', 'ORISPL', 'GENID', 'NUMBLR',
       'GENSTAT', 'PRMVR', 'FUELG1', 'NAMEPCAP', 'CFACT', 'GENNTAN', 'GENNTOZ',
       'GENERSRC', 'GENYRONL', 'GENYRRET', 'orispl_unit'],
      dtype='object')

In [315]:
egrid_gen.loc[(egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit']))].groupby('FUELG1').sum()

,SEQGEN19,YEAR,ORISPL,NUMBLR,NAMEPCAP,CFACT,GENNTAN,GENNTOZ,GENYRONL,GENYRRET
FUELG1,,,,,,,,,,
BIT,5383,2019,527,1,11.5,0.114,1.144800e+04,1.144800e+04,0.0,2019.0
JF,7797,6057,18633,0,223.5,0.006,4.251000e+03,1.760010e+02,5934.0,0.0
NG,403604,92874,556767,33,8832.3,9.921,6.190027e+06,3.687363e+06,73010.0,86964.0
NUC,5029,4038,12198,2,2323.0,1.588,1.616538e+07,8.060445e+06,3971.0,4049.0
RC,38197,4038,4902,2,924.0,1.176,4.715981e+06,2.059040e+06,3958.0,4044.0
SUB,220124,38361,68823,19,6506.5,9.869,3.121425e+07,1.318810e+07,37539.0,26308.0


In [322]:
gd_short.df[gd_short.df['orispl_unit'].isin(egrid_gen.loc[(egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values))&(egrid_gen['FUELG1']=='BIT')]['orispl_unit'].values)]['mw'].sum()


85.0

In [321]:
gd_short.df[gd_short.df['orispl_unit'].isin(egrid_gen.loc[(egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values))&(egrid_gen['FUELG1']=='JF')]['orispl_unit'].values)]['mw'].sum()

0.0

In [320]:
gd_short.df[gd_short.df['orispl_unit'].isin(egrid_gen.loc[(egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values))&(egrid_gen['FUELG1']=='SUB')]['orispl_unit'].values)]['mw'].sum()

6350.0

In [323]:
gd_short.df[gd_short.df['orispl_unit'].isin(egrid_gen.loc[(egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values))&(egrid_gen['FUELG1']=='NUC')]['orispl_unit'].values)]['mw'].sum()

0.0

In [324]:
gd_short.df[gd_short.df['orispl_unit'].isin(egrid_gen.loc[(egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values))&(egrid_gen['FUELG1']=='RC')]['orispl_unit'].values)]['mw'].sum()

950.0

In [325]:
gd_short.df[gd_short.df['orispl_unit'].isin(egrid_gen.loc[(egrid_gen['orispl_unit'].isin(unit_drops['orispl_unit'].values))&(egrid_gen['FUELG1']=='NG')]['orispl_unit'].values)]['mw'].sum()

7361.0

In [326]:
6350+950+85

7385

### Missing (since only covering US): 
- HR Milner in Alberta
- Genesee in Alberta
- Sheerness in Alberta

